In [1]:
# Imports
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Lambda, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.utils.vis_utils import plot_model
from tensorflow.keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

2023-04-10 11:12:18.900246: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-10 11:12:34.022927: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-10 11:12:34.022998: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-10 11:12:34.770528: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-10 11:12:55.188025: W tensorflow/stream_executor/platform/de

In [2]:
file = str('target.dat')
f = open(file,'r')
linea = f.readline()
fields = linea.split()
dim = int(fields[0])
target = np.ndarray(shape=(int(dim)))
print(target.shape)
for i in range(dim):
    linea = f.readline()
    fields = linea.split()
    target[i] = float(fields[0])
f.close()

(1659384,)


In [3]:
file = str('data.dat')
f = open(file,'r')
linea = f.readline()
fields = linea.split()
dim = int(fields[0])
data = np.ndarray(shape=(int(dim),9))
print(data.shape)
for i in range(dim):
    linea = f.readline()
    fields = linea.split()
    for j in range(9):
        data[i][j] = float(fields[j])
f.close()

(1659384, 9)


In [4]:
local_input = np.ndarray(shape=(int(target.shape[0]),4))
nonlocal_input = np.ndarray(shape=(int(target.shape[0]),25))
for i in range(int(target.shape[0])):
    for j in range(4):
        local_input[i][j] = data[i][j+1]
    for j in range(25):
        nonlocal_input[i][j] = np.exp(-0.5*np.power(data[i][0]-0.5*float(j),2))

In [5]:
def shifted_sofplus(x):
    return tf.math.log(0.5*tf.math.exp(x)+0.5)

In [6]:
def local_model():
    
    input = Input(shape=(4,))
    x = Dense(400,activation='relu')(input)
    x = Dense(200,activation='relu')(x)
    x = Dense(100,activation='relu')(x)
    x = Dense(50,activation='relu')(x)
    x = Dense(25,activation='sigmoid')(x)
    
    return Model(inputs=input,outputs=x)

def nonlocal_model():
    
    input = Input(shape=(25,))
    x = Dense(400)(input)
#    x = Dense(400,activation='relu')(input)
    x = Lambda(shifted_sofplus)(x)
    x = Dense(200)(x)
#    x = Dense(200,activation='relu')(x)
    x = Lambda(shifted_sofplus)(x)
    x = Dense(100)(x)
#    x = Dense(100,activation='relu')(x)
    x = Lambda(shifted_sofplus)(x)
    x = Dense(50)(x)
#    x = Dense(50,activation='relu')(x)
    x = Lambda(shifted_sofplus)(x)
    x = Dense(25)(x)
#    x = Dense(25,activation='sigmoid')(x)
    x = Lambda(shifted_sofplus)(x)
    
    return Model(inputs=input, outputs=x)

def final_model():
    
    input = Input(shape=(25,))
    x = Dense(400,activation='relu')(input)
    x = Dense(200,activation='relu')(x)
    x = Dense(100,activation='relu')(x)
    x = Dense(50,activation='relu')(x)
    x = Dense(25,activation='relu')(x)
    x = Dense(1,activation='sigmoid')(x)
    
    return Model(inputs=input, outputs=x)

In [7]:
local_network = local_model()
nonlocal_network = nonlocal_model()
final_network = final_model()

2023-04-10 11:15:45.814807: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-10 11:15:45.814905: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-10 11:15:45.814997: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (lrincon-Latitude-5490): /proc/driver/nvidia/version does not exist
2023-04-10 11:15:45.817375: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
input_local = Input(shape=(4,))
vect_local = local_network(input_local)

input_nonlocal = Input(shape=(25,))
vect_nonlocal = nonlocal_network(input_nonlocal)

features = tf.math.multiply(vect_local,vect_nonlocal)
KED = final_network(features)

model = Model(inputs=[input_local,input_nonlocal],outputs=KED)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 25)]         0           []                               
                                                                                                  
 model (Functional)             (None, 25)           108625      ['input_5[0][0]']                
                                                                                                  
 model_1 (Functional)           (None, 25)           117025      ['input_6[0][0]']                
                                                                                            

In [10]:
model.compile(optimizer='adam',loss='mape',metrics=['mae'])

In [11]:
history = model.fit([local_input,nonlocal_input],target,epochs=100,batch_size=1024)

2023-04-10 11:16:05.733127: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 26550144 exceeds 10% of free system memory.
2023-04-10 11:16:06.055223: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 165938400 exceeds 10% of free system memory.


Epoch 1/100
 426/1621 [======>.......................] - ETA: 18s - loss: 290535.4375 - mae: 0.1127

KeyboardInterrupt: 

In [9]:
model.save("KED_model.h5",overwrite=True)

25928/25928 [==============================] - 17s 668us/step - loss: 0.0815 - mae: 7.0093e-05
